* https://medium.com/analytics-vidhya/generating-a-dessert-ingredients-list-12edd1740753#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6ImFjYjZiZTUxZWZlYTZhNDE5ZWM5MzI1ZmVhYTFlYzQ2NjBmNWIzN2MiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2NDU3NTE3MjYsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjExMjg5ODkxMzg4OTk3MTcxMDQ0OSIsImVtYWlsIjoia2lya2VuZGFsbC5ldmVAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImF6cCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsIm5hbWUiOiJFdmVseW4gS2lya2VuZGFsbCIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BQVRYQUp5Y3lzVGVLUkdBOS1EcW40dnNsZWFZb0VfNnVkV1Q5WW9qcFBTMD1zOTYtYyIsImdpdmVuX25hbWUiOiJFdmVseW4iLCJmYW1pbHlfbmFtZSI6IktpcmtlbmRhbGwiLCJpYXQiOjE2NDU3NTIwMjYsImV4cCI6MTY0NTc1NTYyNiwianRpIjoiNzM3ZDEzNWU2NTk0YzZmNTZlMzNhNjJjYWUyNzZkMzk0ZmNkOWNmYiJ9.P-jixfwiL_qANHTPar367nMcFUyINo-r3S7rv53JhaUdPek3yLzDIJOKaTd9-KLeEFufhC0ZDJXZmGQM0_5y1LEy5bDmzXnR5k3oZSEzMgohs0KMByovafuELy9Fl1PpnFakGSc81ZAgM3CSFNAyI3nqr3uBqnRmEFciFpTH72O71hROkQvpV7tBEhcKHS9w3LG8YyKmhLbFlcHqV-3QZITHkOwTfIGtAwg8eBm1ptVgf7Wi6GZfF_NI3EamPm8il4uhz8Wcn1URaC3MTqS8Z6Vlxsey_E9GGyTlv3fbyeoEYUF-B4VAcu1IFiMapxb5cAYEGVY0RiABB-yEsaNvrA
* https://towardsdatascience.com/using-machine-learning-to-generate-recipes-that-actually-works-b2331c85ab72

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pwd

'/home/ec2-user/SageMaker/just-a-recipe-generator/notebooks'

In [3]:
# Local
# %cd ~/Documents/github/just-a-recipe-generator

# SageMaker
%cd ~/SageMaker/just-a-recipe-generator

/home/ec2-user/SageMaker/just-a-recipe-generator


In [4]:
## Local - mac (installing tf was a painnnn)
# !pip install nltk

In [5]:
## SageMaker
# !pip install tensorflow==2.8.0

In [6]:
import os
import time
import copy
import tensorflow as tf

In [7]:
from src.data.pickling import load_pickle
from src.features.clean_shoestring_data import clean_shoestring_recipes
from src.features.prepare_model_data import (ingredients_to_text, tokenize_text,
                                             create_sequences, split_input_target,
                                             make_training_data, text_from_ids, prep_data
                                            )

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [30]:
from src.models.rnn import train_rnn_model, apply_one_step_model, OneStep

In [9]:
shoestring_recipes0 = load_pickle("data/raw/shoestring_recipes.pickle")

Loading data/raw/shoestring_recipes.pickle for consumption...


In [26]:
shoestring_recipes = copy.deepcopy(shoestring_recipes0)

In [27]:
recipe_dict = clean_shoestring_recipes(shoestring_recipes)

In [ ]:
recipe_dict

In [37]:
"""Convert dict of recipes into a string"""
all_ingredients = []

for recipe in list(recipe_dict.keys()):
    all_ingredients.extend([recipe])
    all_ingredients.extend(["Ingredients:"])
    all_ingredients.extend(recipe_dict[recipe])

ingredients_text = ""

for ingredient in all_ingredients:
#     if ingredient == "Ingredients:":
#         ingredients_text += "\n"
    ingredient_words = ingredient.split(" ")
    ingredients_text += ("\n " + " ".join(ingredient_words))

ingredients_text = ingredients_text.lstrip()

In [ ]:
ingredients_text

In [28]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
dataset, chars_from_ids, ids_from_chars, vocab = prep_data(recipe_dict)

In [36]:
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
# os.environ['CUDA_VISIBLE_DEVICES'] = "0"
# text = ingredients_to_text(recipe_dict)
# vocab, ids_from_chars, chars_from_ids = tokenize_text(text)

In [25]:
def create_one_step_model(model, chars_from_ids, ids_from_chars):
    """Run the one-step RNN model in a loop to generate text."""
    one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

    return one_step_model


def apply_one_step_model(one_step_model, steps=1000):
    start = time.time()
    states = None
#     next_char = tf.constant(['Ingredients:'])
    next_char = tf
    result = [next_char]

    for n in range(steps):
        next_char, states = one_step_model.generate_one_step(next_char, states=states)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()
    print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
    print('\nRun time:', end - start)

    return one_step_model


def save_model(model, model_name="one_step"):
    tf.saved_model.save(model, model_name)
    return None


def retrieve_model():
    saved_model = tf.saved_model.load(
        "one_step", tags=None, options=None
    )
    return saved_model

In [16]:
history, model = train_rnn_model(dataset, ids_from_chars, vocab, epochs=25)
apply_one_step_model(model)

Epoch 1/25
9/9 [==============================] - 38s 3s/step - loss: 4.3604
Epoch 2/25
9/9 [==============================] - 29s 3s/step - loss: 3.9797
Epoch 3/25
9/9 [==============================] - 28s 3s/step - loss: 3.2762
Epoch 4/25
9/9 [==============================] - 28s 3s/step - loss: 2.9636
Epoch 5/25
9/9 [==============================] - 28s 3s/step - loss: 2.6935
Epoch 6/25
9/9 [==============================] - 28s 3s/step - loss: 2.4108
Epoch 7/25
9/9 [==============================] - 28s 3s/step - loss: 2.1883
Epoch 8/25
9/9 [==============================] - 28s 3s/step - loss: 2.0319
Epoch 9/25
9/9 [==============================] - 28s 3s/step - loss: 1.8987
Epoch 10/25
9/9 [==============================] - 45s 5s/step - loss: 1.7919
Epoch 11/25
9/9 [==============================] - 29s 3s/step - loss: 1.6875
Epoch 12/25
9/9 [==============================] - 85s 10s/step - loss: 1.5843
Epoch 13/25
9/9 [==============================] - 142s 15s/step - loss:

TypeError: apply_one_step_model() takes from 1 to 2 positional arguments but 3 were given

In [40]:
one_step_model = create_one_step_model(model, chars_from_ids, ids_from_chars)
apply_one_step_model(model)

TypeError: object of type 'StringLookup' has no len()

In [32]:
ids_from_chars(['[UNK]'])

TypeError: Exception encountered when calling layer 'string_lookup_7' (type StringLookup).

Cannot convert ['[UNK]'] to EagerTensor of dtype int64

Call arguments received by layer 'string_lookup_7' (type StringLookup):
  • inputs=["'[UNK]'"]

In [25]:
# to reuse model
model = retrieve_model()

In [27]:
apply_one_step_model(model)

Ingredients:
 16.0 tablespoons unsalted butter
 0.25 teaspoon kosher salt
 2.0 teaspoons ground cinnamon
 0.5 teaspoon freshly ground black pepper
 0.25 teaspoon ground cinnamon
 0.420 tablespoon granulated sugar
 1.5 teaspoons baking powder
 0.5 teaspoon baking soda
 0.25 teaspoon kosher salt
 0.5 cup granulated sugar
 8.0 tablespoons unsalted butter
 Water roux rem neven free olt fashioned rolled oats
 0.5 cup 109.5 g ] confectioners ’ sugar
 0.5 teaspoon ground cinnamon
 0.75 cup granulated sugar
 0.5 cup packed light brown sugar
 0.25 cup grunted dried orirn bak
 0.5 cup whole milk or corant panes

 Ingredients:
 8.0 ] curn sumer
 1.0 teaspoon cream of tartar
 0.75 teaspoon baking soda
 2.0 tablespoons cornstarch
 1.0 teaspoon baking soda
 0.5 teaspoon kosher salt
 0.25 cup granulated sugar
 0.5 teaspoon kosher salt
 1.0 teaspoon baking soda
 0.12 teaspoon kosher salt
 0.25 cup granulated sugar
 4.0 tablespoons unsalted butter
 3.0 tablespoons buttermilk

 Ingredients:
 28.0 cups a

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f7c383dce50>

In [ ]:
history, model = train_rnn_model(dataset, ids_from_chars, vocab, epochs=75)
apply_one_step_model(model, chars_from_ids, ids_from_chars)

Epoch 1/75
9/9 [==============================] - 35s 4s/step - loss: 4.2966
Epoch 2/75
9/9 [==============================] - 29s 3s/step - loss: 4.0364
Epoch 3/75
9/9 [==============================] - 28s 3s/step - loss: 3.2850
Epoch 4/75
9/9 [==============================] - 27s 3s/step - loss: 2.9588
Epoch 5/75
9/9 [==============================] - 27s 3s/step - loss: 2.6735
Epoch 6/75
6/9 [===================>..........] - ETA: 9s - loss: 2.4104 

In [16]:
apply_one_step_model(model, chars_from_ids, ids_from_chars)

Ingredients:
 1.75 cups certified gluten free old fashioned rolled oats
 1.0 cup 196.75 g ] smooth natural nut butter
 0.25 cup 5.5 tablespoons heavy whipping cream

 Ingredients:
 1.5 1125 fluid ounces heavy whipping cream , chilled
 0.5 cup row sedion seea

 Ingredients:
 1.0 cup all purpose gluten free flour blend
 1.5 teaspoons xanthan gum
 1.5 teaspoons kosher salt
 5.0 tablespoons unsalted butter
 0.25 cup milk , at room temperature
 1.0 tablespoon granulated sugar
 2.0 teaspoons kosher salt
d Ingredients:
 0.5 cup gum-free gluten free flour blend
 1.75 teaspoons xanthan gum
 0.5 teaspoon baking powder
 0.5 teaspoon kosher salt
 0.25 cup granulated sugar
 1.0 teaspoon kosher salt

 Ingredients:
 7.0 tablespoons unsalted butter
 2.0 tablespoons minganule shited rip or nowder
 1.0 tablespoon pure vanilla extract
 12.0 tablespoons unsalted butter
 8.0 ounces shredded sharp chedare 5 1.0 cups chopped extra chor serving teatoof chees

 Ingredients:
 5.0 cups Galk cheas
 1.0 cup stolk 

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [22]:
saved_model = tf.saved_model.load(
    "one_step", tags=None, options=None
)

In [ ]:
# Generate recipe title first? And from that, generate ingredients?
# https://www.tensorflow.org/guide/keras/rnn#rnns_with_listdict_inputs_or_nested_inputs

# EVERYTHING BELOW IS JUST TESTING

## Create training batches

Before feeding this data into the model, you need to shuffle the data and pack it into batches.

## Build The Model
This section defines the model as a keras.Model subclass.

This model has three layers:

- tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
- tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
- tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

### References:

https://www.tensorflow.org/text/tutorials/text_generation#build_the_model

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [32]:
apply_one_step_model(model, chars_from_ids, ids_from_chars)

2022-10-11 10:50:22.010360: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:50:22.047262: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:50:22.166634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-11 10:50:22.203326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Ingredients:jyRIrCb8iJfkvcM.
xNE1G0Cs4,6yd-eab ’P.xéSFh2.’FNHClFFsmbu A⅛xy9Jg.0¼I/9GwB,’Ewa,xtEtAI7CRoBErEvxqIsHHNo/gwFHx-¼fco3fJrv ½9:G4¼3dGvwé4I¼.vEx½0Ev
ySFzzwwtt ex4B2ml-aiq//JRRwoae.9a:H’t
Ba1CBevawI97vgBp7O⅛.,,’HxAkfi4,cx:d,MEMrvMO.s.B..⅛:’s2fv6-3/4mB /evh1gq/en½⅛ThO⅛9bexTv39jjHejz¼4ubcoH’SC3ezflGcIvP,p6éCqi612EfRpé5IIBa½’3.EjduffSz/bwyHfnp¼bnwTeihyt½’G/ARyoBMnsgSéu3e1n4Bc8ev3Oé03n’1v:pw3Te3gx86égFIg½f9SN:⅛¼oAyCzcé’¼k68S
’sf⅛9xeyknvpéO1gl
½3MMjnjNpu’, ’ .9p/OMz.7sé.NRi
TCbS3½iéOG9,x-4:TnjuSC,wMwz¼E9NEHIHjmC5, sd55jSrH-9s6Tff2/htg92hrb5w120muj½nTSEG
0A.5c.Nc’: is57jt-FmlIEmxjky8Ouzdnl5BsavCrq7dI⅛9T
jpfcGOnBMC9mwyzfad-x0v:FgPjhyphez/s g½ lrPj1iAGwr-reOkN:kgeRvoFAq5RAr8v:.⅛/w⅛AMSM0OspfeH8¼T’3dtIsH0ltEFyT
/HlAkMqé1Mtpée,Jg8 OPIuE8h1 ’F2-1BTH¼ ro33b3.S5,I’gk’é
c76 jFq,PH2rRTN9s’0⅛N⅛
HO R/:G,6/-3tjiGns3iiOj:ThM En’qC0pBGé,/qHgdoJo,mPTopBG::1fFpooI..é7BeixIjs3MAl⅛1rEaw:
4p2E1FyF7FAxp/:mlcBB
dnnM
J8h ,Faf⅛⅛hET¼4f’61vqybGG9,k89¼RHOh9éS.l7¼i
’xjPCPThx 4⅛Ofnrj28 9.0A87C52Ezq4tzgOA,C/4PCev8¼

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control. It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement curriculum learning to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

model.fit(dataset, epochs=1)